# side model

In [1]:
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

df_features = pd.read_parquet("data/features_6h.parquet")
df_features = df_features[SIDE_ALL]
label_side = np.load("data/label_side.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(7442, 2500)
(7442, 7)


24h_dft_dom_cycle_dt_lag39    352
24h_dft_dom_cycle_lag39       348
24h_dft_dom_cycle_dt_lag38    348
24h_dft_dom_cycle_dt_lag37    344
24h_dft_dom_cycle_lag38       344
                             ... 
6h_comb_spectrum_pwr_34         0
6h_bandpass                     0
6h_mod_rsi                      0
6h_comb_spectrum_pwr_32         0
6h_stc                          0
Length: 2500, dtype: int64

In [2]:
df_features = df_features[352:]
label_side = label_side[352:]

print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(7090, 2500)
(7090, 7)


90m_acc_swing_index                   0
24h_comb_spectrum_pwr_29              0
24h_comb_spectrum_pwr_11              0
24h_comb_spectrum_pwr_2               0
24h_comb_spectrum_pwr_24              0
                                     ..
6h_evenbetter_sinewave_short_lag12    0
6h_evenbetter_sinewave_short_lag13    0
6h_evenbetter_sinewave_short_lag14    0
6h_evenbetter_sinewave_short_lag15    0
24h_williams_r_lag9                   0
Length: 2500, dtype: int64

In [3]:
side_features = df_features
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(7090, 2500)
(array([0, 1]), array([3132, 3958]))
(array([0, 1]), array([577, 841]))


In [4]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-26 21:49:00,032] A new study created in memory with name: no-name-933500a2-06c1-4e75-8d4e-fea97b03d9f9
[I 2025-03-26 21:49:01,879] Trial 0 finished with value: 0.8234646795409444 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 254, 'max_depth': 548, 'min_gain_to_split': 0.9182076814269632, 'min_data_in_leaf': 126, 'lambda_l1': 82.72916933564703, 'lambda_l2': 89.13609293244696, 'num_boost_round': 631}. Best is trial 0 with value: 0.8234646795409444.


precision: 0.8047858942065491, recall: 0.7598097502972652, f1: 0.781651376146789


[I 2025-03-26 21:49:04,641] Trial 1 finished with value: 0.8302631389140187 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 77, 'max_depth': 140, 'min_gain_to_split': 0.08025064061840573, 'min_data_in_leaf': 105, 'lambda_l1': 44.646887060081376, 'lambda_l2': 65.47693048366588, 'num_boost_round': 972}. Best is trial 1 with value: 0.8302631389140187.


precision: 0.8106060606060606, recall: 0.7633769322235434, f1: 0.7862829148805879


[I 2025-03-26 21:49:05,769] Trial 2 finished with value: 0.8299282648163757 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 130, 'max_depth': 355, 'min_gain_to_split': 0.020897157661246314, 'min_data_in_leaf': 196, 'lambda_l1': 72.01798162438553, 'lambda_l2': 83.30665148881813, 'num_boost_round': 232}. Best is trial 1 with value: 0.8302631389140187.


...

precision: 0.8090452261306532, recall: 0.7657550535077289, f1: 0.786805131337813


[I 2025-03-26 21:55:23,297] Trial 98 finished with value: 0.8264239361822703 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 140, 'max_depth': 613, 'min_gain_to_split': 0.7522605838886369, 'min_data_in_leaf': 147, 'lambda_l1': 77.36773628929693, 'lambda_l2': 76.25169425618448, 'num_boost_round': 793}. Best is trial 57 with value: 0.8463226702551432.


precision: 0.7877358490566038, recall: 0.7942925089179548, f1: 0.7910005920663115


[I 2025-03-26 21:55:25,890] Trial 99 finished with value: 0.8425535334884402 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 168, 'max_depth': 695, 'min_gain_to_split': 0.8076649275627319, 'min_data_in_leaf': 185, 'lambda_l1': 71.4489059585805, 'lambda_l2': 88.95430629453304, 'num_boost_round': 900}. Best is trial 57 with value: 0.8463226702551432.


precision: 0.805115712545676, recall: 0.7859690844233056, f1: 0.7954271961492179


In [5]:
study.best_params

{'is_unbalance': True,
 'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 186,
 'max_depth': 522,
 'min_gain_to_split': 0.6790058636917701,
 'min_data_in_leaf': 350,
 'lambda_l1': 65.29591980380066,
 'lambda_l2': 56.83328288489591,
 'num_boost_round': 1301}

In [6]:
import lightgbm as lgb

params = {
    # "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}


dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

prod_model = lgb.train(params, lgb.Dataset(side_features, side_label))

In [7]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
import plotly.express as px
from sklearn.metrics import roc_auc_score

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8463226702551432


In [9]:
model.save_model("model/model_side.txt")
prod_model.save_model("model/model_side_prod.txt")

# meta model

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import META_ALL, SIDE_ALL

df_features = pd.read_parquet("data/features_6h.parquet")
meta_label = np.load("data/label_meta.npy")

model_side = lgb.Booster(model_file="model/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features = df_features[META_ALL]
df_features.isna().sum(axis=0).sort_values(ascending=False)

24h_dft_dom_cycle_dt_lag39    352
24h_dft_dom_cycle_dt_lag38    348
24h_dft_dom_cycle_dt_lag36    340
24h_dft_dom_cycle_dt_lag30    316
24h_dft_dom_cycle_dt_lag23    288
                             ... 
6h_comb_spectrum_pwr_4          0
6h_comb_spectrum_pwr_5          0
6h_comb_spectrum_pwr_6          0
6h_comb_spectrum_pwr_7          0
side_model_res                  0
Length: 2500, dtype: int64

In [3]:
df_features = df_features[352:]
meta_label = meta_label[352:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(7090, 2500)
(7090, 7)


90m_ac_14                            0
24h_dual_diff_dt_lag33               0
24h_dual_diff_dt_lag13               0
24h_dual_diff_dt_lag2                0
24h_dual_diff_dt_lag28               0
                                    ..
6h_evenbetter_sinewave_long_lag12    0
6h_evenbetter_sinewave_long_lag13    0
6h_evenbetter_sinewave_long_lag14    0
6h_evenbetter_sinewave_long_lag15    0
side_model_res                       0
Length: 2500, dtype: int64

In [4]:
meta_features = df_features
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

print(np.unique(meta_label_train, return_counts=True))
print(np.unique(meta_label_test, return_counts=True))

(7090, 2500)
(array([0, 1]), array([ 676, 4996]))
(array([0, 1]), array([ 335, 1083]))


In [5]:
import optuna
from sklearn.metrics import (
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.2, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba, average="weighted")
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return fbeta


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-26 22:04:11,553] A new study created in memory with name: no-name-06e2f45e-2e31-4a57-9cbc-fa93c6b3ea37
[I 2025-03-26 22:04:13,190] Trial 0 finished with value: 0.697162199576821 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 410, 'max_depth': 68, 'min_gain_to_split': 0.40826455629083036, 'min_data_in_leaf': 211, 'lambda_l1': 70.52322788774678, 'lambda_l2': 45.36788812468234, 'num_boost_round': 146}. Best is trial 0 with value: 0.697162199576821.


precision: 0.8853658536585366, recall: 0.6703601108033241, f1: 0.7630057803468208, auc: 0.7569107371728614, fbeta: 0.697162199576821


[I 2025-03-26 22:04:18,118] Trial 1 finished with value: 0.7413108635129649 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 140, 'max_depth': 421, 'min_gain_to_split': 0.980134890476224, 'min_data_in_leaf': 216, 'lambda_l1': 22.186961094385946, 'lambda_l2': 62.43612483404513, 'num_boost_round': 1947}. Best is trial 1 with value: 0.7413108635129649.


precision: 0.8377609108159393, recall: 0.8153277931671283, f1: 0.8263921385119326, auc: 0.758867711304971, fbeta: 0.7413108635129649


[I 2025-03-26 22:04:20,468] Trial 2 finished with value: 0.7160251834962683 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 297, 'max_depth': 556, 'min_gain_to_split': 0.31626519922318014, 'min_data_in_leaf': 219, 'lambda_l1': 81.9728833853199, 'lambda_l2': 30.2697132533943, 'num_boost_round': 632}. Best is trial 1 with value: 0.7413108635129649.


...

precision: 0.8476286579212916, recall: 0.775623268698061, f1: 0.8100289296046287, auc: 0.7669243808657543, fbeta: 0.7303845081170828


[I 2025-03-26 22:12:40,668] Trial 98 finished with value: 0.7420360114781264 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 106, 'max_depth': 693, 'min_gain_to_split': 0.4370585027634205, 'min_data_in_leaf': 204, 'lambda_l1': 31.976662104436464, 'lambda_l2': 77.78308691290499, 'num_boost_round': 1794}. Best is trial 93 with value: 0.7522851366726392.


precision: 0.8403441682600382, recall: 0.8116343490304709, f1: 0.8257397839361202, auc: 0.7719436060693761, fbeta: 0.7420360114781264


[I 2025-03-26 22:12:45,418] Trial 99 finished with value: 0.7390958089244443 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 137, 'max_depth': 628, 'min_gain_to_split': 0.4052902814001208, 'min_data_in_leaf': 140, 'lambda_l1': 33.621636410591094, 'lambda_l2': 80.104720129506, 'num_boost_round': 1869}. Best is trial 93 with value: 0.7522851366726392.


precision: 0.844574780058651, recall: 0.7977839335180056, f1: 0.8205128205128205, auc: 0.7647854908284064, fbeta: 0.7390958089244443


In [6]:
study.best_params

{'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 105,
 'max_depth': 234,
 'min_gain_to_split': 0.5575318693557914,
 'min_data_in_leaf': 167,
 'lambda_l1': 29.2566471043314,
 'lambda_l2': 77.68240276981057,
 'num_boost_round': 747}

In [7]:
import lightgbm as lgb

params = {
    "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [8]:
import plotly.express as px

res = model.predict(meta_features_test)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
model.save_model("model/model_meta.txt")
prod_model.save_model("model/model_meta_prod.txt")